<a href="https://colab.research.google.com/github/alyssarose05/detained-child-migrants/blob/main/Combined_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up detained migrant dataset

## Set up detained children database

In [1]:
# Import pandas and SQL
import pandas as pd
import sqlite3

### Initialize the database

In [2]:
# Created detained_children database
con = sqlite3.connect("detained_children.db")

### Upload migrant and continent data to `detained_children`

In [3]:
# Upload NYTimes data to database
children_df = pd.read_csv("data.csv")
children_df.to_sql("Migrants", con, if_exists="replace", index=False)

/tmp/ipython-input-3623372180.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  children_df = pd.read_csv("data.csv")


553322

In [4]:
# Upload continents to database
continent_df = pd.read_csv("continents.csv")
continent_df.to_sql("Continents", con, if_exists="replace", index=False)

196

### Modify table columns

The column names should be in `snake_case`, so that they're optimized for SQL queries. Also, the corresponding country and continent columns should have identical names, so that `Migrants` and `Continents` can be joined.

In [5]:
# Rename columns in migrant data
modified_migrant_df = pd.read_sql_query("""
    SELECT `ID` AS id,
        `Child's Country of Origin` AS country,
        `Child's Gender` AS gender,
        `Child's Date of Entry` AS entry_date,
        `Child's Date of Release` AS release_date,
        `Sponsor Zipcode` AS  sponsor_zipcode,
        `Sponsor Category` AS sponsor_category,
        `Relationship of Sponsor` AS sponsor_relationship
    FROM Migrants
""", con)

# Replace the `Migrants` table with modified column data
modified_migrant_df.to_sql("Migrants", con, if_exists="replace", index=False)

553322

In [6]:
# Rename columns in continent data
modified_continent_df = pd.read_sql_query("""
    SELECT `Country` AS country,
        `Continent` AS continent
    FROM Continents
""", con)

# Replace the `Continents` table with modified column data
modified_continent_df.to_sql("Continents", con, if_exists="replace", index=False)

196

## Retrieve final dataframe

### Join `Migrants` and `Continents`

Joining `Migrants` and `Continents` based on the `country` column will allow us to get the `continent` of each child migrant.

In [7]:
final_df = pd.read_sql_query("""
  SELECT a.country, b.continent, gender, entry_date, release_date, sponsor_zipcode, sponsor_category, sponsor_relationship
  FROM Migrants a
  LEFT JOIN Continents b
  ON a.country = b.country;
""", con)

### Save and close the database

The result of the SQL query above will be used for further analysis with Tableau.

In [8]:
# Save dataframe as CSV file
final_df.to_csv("final_df.csv", index=False)

In [9]:
# Close database connection
con.close()